## gcp code, improve from nlp 07

In [ ]:
import torch
import neptune
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import json
import os
from sklearn.metrics import accuracy_score, f1_score
from rouge_score import rouge_scorer
import subprocess
import matplotlib.pyplot as plt

# Inisialisasi Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),  # Gunakan environment variable untuk keamanan
    name="Fine-tuning Nusantara Chat Model",
    tags=["fine-tuning", "NLP", "Transformers"],
    dependencies="infer",
    capture_hardware_metrics=True,
)

# Fungsi untuk mendownload dataset dari bucket GCS
def download_from_gcs(bucket_path, local_path):
    subprocess.run(['gsutil', 'cp', bucket_path, local_path])

# Fungsi untuk mengupload model ke bucket GCS
def upload_to_gcs(local_path, bucket_path):
    subprocess.run(['gsutil', 'cp', '-r', local_path, bucket_path])

# Menggunakan GPU jika tersedia, fallback ke CPU jika OutOfMemoryError terjadi
def load_model(model_name):
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
        print("Model loaded on GPU" if torch.cuda.is_available() else "Model loaded on CPU")
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print("Out of memory error on GPU. Loading model on CPU...")
            model = AutoModelForCausalLM.from_pretrained(model_name).to('cpu')
        else:
            raise e
    return model

# Load dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi evaluasi ROUGE dan F1 Score
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids

    # Decode predictions dan labels menjadi teks
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Hitung F1 Score
    f1 = f1_score(decoded_labels, decoded_preds, average='weighted')

    # Hitung ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {key: 0 for key in scorer.score("", "").keys()}
    n = len(decoded_preds)

    for i in range(n):
        scores = scorer.score(decoded_labels[i], decoded_preds[i])
        for key in rouge_scores:
            rouge_scores[key] += scores[key].fmeasure

    # Ambil rata-rata ROUGE score
    rouge_scores = {key: value / n for key, value in rouge_scores.items()}

    # Log metrik ke Neptune
    run["metrics/f1"].append(f1)
    run["metrics/rouge1"].append(rouge_scores['rouge1'])
    run["metrics/rouge2"].append(rouge_scores['rouge2'])
    run["metrics/rougeL"].append(rouge_scores['rougeL'])

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1,
        "rouge1": rouge_scores['rouge1'],
        "rouge2": rouge_scores['rouge2'],
        "rougeL": rouge_scores['rougeL']
    }

# Modifikasi fungsi fine_tune_model untuk menyertakan tokenizer sebagai global variable
def fine_tune_model(model_name, dataset_file, output_dir):
    global tokenizer
    # Load model dan tokenizer
    model = load_model(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load dataset
    data = load_dataset(dataset_file)

    # Preprocess dataset
    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    # Logging hyperparameter ke Neptune
    params = {
        "model_name": model_name,
        "batch_size": 10,
        "num_train_epochs": 16,
        "learning_rate": 0.0005,
        "gradient_accumulation_steps": 16,
    }
    run["parameters"] = params

    # Mendefinisikan argumen training
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=params["batch_size"],
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        num_train_epochs=params["num_train_epochs"],
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Menggunakan fungsi compute_metrics yang baru
    )

    # Jalankan pelatihan
    trainer.train()

    # Simpan model dan tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Hentikan logging Neptune
    run.stop()

    print(f"Model fine-tuning selesai dan disimpan di: {output_dir}")

# Contoh penggunaan
bucket_dataset_path = 'gs://pintarpath-trying/output2.json'  # Path di GCS
local_dataset_file = './output2.json'  # Path lokal untuk dataset
bucket_output_dir = 'gs://pintarpath-trying/saved_model/fine-tuned-model'  # Path output di GCS
local_output_dir = './fine-tuned-model'  # Path lokal untuk menyimpan model sebelum upload

# Download dataset dari GCS
download_from_gcs(bucket_dataset_path, local_dataset_file)

# Fine-tuning model
fine_tune_model(model_name="kalisai/Nusantara-0.8b-Indo-Chat", dataset_file=local_dataset_file, output_dir=local_output_dir)

# Upload model ke GCS (Dikomentari untuk menghemat penyimpanan)
# upload_to_gcs(local_output_dir, bucket_output_dir)


## Improve for Logging Visualisasi

In [ ]:
import torch
import neptune
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import json
import os
from sklearn.metrics import accuracy_score, f1_score
from rouge_score import rouge_scorer
import subprocess
import matplotlib.pyplot as plt

# Inisialisasi Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),  # Gunakan environment variable untuk keamanan
    name="Fine-tuning Nusantara Chat Model",
    tags=["fine-tuning", "NLP", "Transformers"],
    dependencies="infer",
    capture_hardware_metrics=True,
)

# Fungsi untuk mendownload dataset dari bucket GCS
def download_from_gcs(bucket_path, local_path):
    subprocess.run(['gsutil', 'cp', bucket_path, local_path])

# Fungsi untuk mengupload model ke bucket GCS
def upload_to_gcs(local_path, bucket_path):
    subprocess.run(['gsutil', 'cp', '-r', local_path, bucket_path])

# Menggunakan GPU jika tersedia, fallback ke CPU jika OutOfMemoryError terjadi
def load_model(model_name):
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
        print("Model loaded on GPU" if torch.cuda.is_available() else "Model loaded on CPU")
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print("Out of memory error on GPU. Loading model on CPU...")
            model = AutoModelForCausalLM.from_pretrained(model_name).to('cpu')
        else:
            raise e
    return model

# Load dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi evaluasi ROUGE dan F1 Score
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids

    # Decode predictions dan labels menjadi teks
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Hitung F1 Score
    f1 = f1_score(decoded_labels, decoded_preds, average='weighted')

    # Hitung ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {key: 0 for key in scorer.score("", "").keys()}
    n = len(decoded_preds)

    for i in range(n):
        scores = scorer.score(decoded_labels[i], decoded_preds[i])
        for key in rouge_scores:
            rouge_scores[key] += scores[key].fmeasure

    # Ambil rata-rata ROUGE score
    rouge_scores = {key: value / n for key, value in rouge_scores.items()}

    # Log metrik ke Neptune
    run["metrics/f1"].append(f1)
    run["metrics/rouge1"].append(rouge_scores['rouge1'])
    run["metrics/rouge2"].append(rouge_scores['rouge2'])
    run["metrics/rougeL"].append(rouge_scores['rougeL'])

    # Membuat plot ROC curve (contoh visualisasi)
    plt.figure(figsize=(10, 7))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(decoded_preds, decoded_labels, label='ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt_fig_roc = plt.gcf()
    run["val/roc_curve"].upload(plt_fig_roc)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1,
        "rouge1": rouge_scores['rouge1'],
        "rouge2": rouge_scores['rouge2'],
        "rougeL": rouge_scores['rougeL']
    }

# Modifikasi fungsi fine_tune_model untuk menyertakan tokenizer sebagai global variable
def fine_tune_model(model_name, dataset_file, output_dir):
    global tokenizer
    # Load model dan tokenizer
    model = load_model(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load dataset
    data = load_dataset(dataset_file)

    # Preprocess dataset
    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    # Logging hyperparameter ke Neptune
    params = {
        "model_name": model_name,
        "batch_size": 10,
        "num_train_epochs": 16,
        "learning_rate": 0.0005,
        "gradient_accumulation_steps": 16,
    }
    run["parameters"] = params

    # Mendefinisikan argumen training
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=params["batch_size"],
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        num_train_epochs=params["num_train_epochs"],
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Menggunakan fungsi compute_metrics yang baru
    )

    # Jalankan pelatihan
    trainer.train()

    # Simpan model dan tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Hentikan logging Neptune
    run.stop()

    print(f"Model fine-tuning selesai dan disimpan di: {output_dir}")

# Contoh penggunaan
bucket_dataset_path = 'gs://pintarpath-trying/output2.json'  # Path di GCS
local_dataset_file = './output2.json'  # Path lokal untuk dataset
bucket_output_dir = 'gs://pintarpath-trying/saved_model/fine-tuned-model'  # Path output di GCS
local_output_dir = './fine-tuned-model'  # Path lokal untuk menyimpan model sebelum upload

# Download dataset dari GCS
download_from_gcs(bucket_dataset_path, local_dataset_file)

# Fine-tuning model
fine_tune_model(model_name="kalisai/Nusantara-0.8b-Indo-Chat", dataset_file=local_dataset_file, output_dir=local_output_dir)

# Upload model ke GCS (Dikomentari untuk menghemat penyimpanan)
# upload_to_gcs(local_output_dir, bucket_output_dir)


# Imrprove

In [ ]:
import torch
import neptune
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import json
import os
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
from rouge_score import rouge_scorer
import subprocess
import matplotlib.pyplot as plt
from torch.nn.functional import softmax

# Inisialisasi Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),  # Gunakan environment variable untuk keamanan
    name="Fine-tuning Nusantara Chat Model",
    tags=["fine-tuning", "NLP", "Transformers"],
    dependencies="infer",
    capture_hardware_metrics=True,
)

# Fungsi untuk mendownload dataset dari bucket GCS
def download_from_gcs(bucket_path, local_path):
    subprocess.run(['gsutil', 'cp', bucket_path, local_path])

# Fungsi untuk mengupload model ke bucket GCS
def upload_to_gcs(local_path, bucket_path):
    subprocess.run(['gsutil', 'cp', '-r', local_path, bucket_path])

# Menggunakan GPU jika tersedia, fallback ke CPU jika OutOfMemoryError terjadi
def load_model(model_name):
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
        print("Model loaded on GPU" if torch.cuda.is_available() else "Model loaded on CPU")
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print("Out of memory error on GPU. Loading model on CPU...")
            model = AutoModelForCausalLM.from_pretrained(model_name).to('cpu')
        else:
            raise e
    return model

# Load dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi evaluasi ROUGE, F1 Score, dan ROC curve
def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)  # Prediksi model dalam bentuk logits
    probabilities = softmax(logits, dim=-1).cpu().numpy()  # Konversi logits ke probabilitas

    # Misal, jika model Anda adalah binary classification
    pred_probs = probabilities[:, 1]  # Probabilitas untuk kelas positif
    labels = pred.label_ids

    # Hitung ROC curve
    fpr, tpr, _ = roc_curve(labels, pred_probs)
    roc_auc = auc(fpr, tpr)

    # Hitung F1 Score
    pred_labels = pred_probs.round()  # Threshold 0.5 untuk binary classification
    f1 = f1_score(labels, pred_labels, average='weighted')

    # Hitung ROUGE (jika relevan, sesuaikan untuk task lain)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {key: 0 for key in scorer.score("", "").keys()}
    n = len(labels)

    for i in range(n):
        scores = scorer.score(str(labels[i]), str(pred_labels[i]))
        for key in rouge_scores:
            rouge_scores[key] += scores[key].fmeasure

    # Ambil rata-rata ROUGE score
    rouge_scores = {key: value / n for key, value in rouge_scores.items()}

    # Log metrik ke Neptune
    run["metrics/f1"].append(f1)
    run["metrics/roc_auc"].append(roc_auc)
    run["metrics/rouge1"].append(rouge_scores['rouge1'])
    run["metrics/rouge2"].append(rouge_scores['rouge2'])
    run["metrics/rougeL"].append(rouge_scores['rougeL'])

    # Plot ROC curve
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt_fig_roc = plt.gcf()
    run["val/roc_curve"].upload(plt_fig_roc)

    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy_score(labels, pred_labels),
        "f1": f1,
        "rouge1": rouge_scores['rouge1'],
        "rouge2": rouge_scores['rouge2'],
        "rougeL": rouge_scores['rougeL']
    }

# Modifikasi fungsi fine_tune_model untuk menyertakan tokenizer sebagai global variable
def fine_tune_model(model_name, dataset_file, output_dir):
    global tokenizer
    # Load model dan tokenizer
    model = load_model(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load dataset
    data = load_dataset(dataset_file)

    # Preprocess dataset
    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    # Logging hyperparameter ke Neptune
    params = {
        "model_name": model_name,
        "batch_size": 12,
        "num_train_epochs": 20,
        "learning_rate": 0.0005,
        "gradient_accumulation_steps": 16,
    }
    run["parameters"] = params

    # Mendefinisikan argumen training
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=params["batch_size"],
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        num_train_epochs=params["num_train_epochs"],
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Menggunakan fungsi compute_metrics yang baru
    )

    # Jalankan pelatihan
    trainer.train()

    # Simpan model dan tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Hentikan logging Neptune
    run.stop()

    print(f"Model fine-tuning selesai dan disimpan di: {output_dir}")

# Contoh penggunaan
bucket_dataset_path = 'gs://pintarpath-trying/output2.json'  # Path di GCS
local_dataset_file = './output2.json'  # Path lokal untuk dataset
bucket_output_dir = 'gs://pintarpath-trying/saved_model/fine-tuned-model'  # Path output di GCS
local_output_dir = './fine-tuned-model'  # Path lokal untuk menyimpan model sebelum upload

# Download dataset dari GCS
download_from_gcs(bucket_dataset_path, local_dataset_file)

# Fine-tuning model
fine_tune_model(model_name="kalisai/Nusantara-0.8b-Indo-Chat", dataset_file=local_dataset_file, output_dir=local_output_dir)

# Upload model ke GCS (Dikomentari untuk menghemat penyimpanan)
# upload_to_gcs(local_output_dir, bucket_output_dir)


In [ ]:
import torch
import neptune
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import json
import os
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
from rouge_score import rouge_scorer
import subprocess
import matplotlib.pyplot as plt
from torch.nn.functional import softmax
from neptune.types import File

# Inisialisasi Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),  # Gunakan environment variable untuk keamanan
    name="Fine-tuning Nusantara Chat Model",
    tags=["fine-tuning", "NLP", "Transformers"],
    dependencies="infer",
    capture_hardware_metrics=True,
)

# Fungsi untuk mendownload dataset dari bucket GCS
def download_from_gcs(bucket_path, local_path):
    subprocess.run(['gsutil', 'cp', bucket_path, local_path])

# Fungsi untuk mengupload file ke bucket GCS
def upload_to_gcs(local_path, bucket_path):
    subprocess.run(['gsutil', 'cp', local_path, bucket_path])

# Menggunakan GPU jika tersedia, fallback ke CPU jika OutOfMemoryError terjadi
def load_model(model_name):
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
        print("Model loaded on GPU" if torch.cuda.is_available() else "Model loaded on CPU")
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print("Out of memory error on GPU. Loading model on CPU...")
            model = AutoModelForCausalLM.from_pretrained(model_name).to('cpu')
        else:
            raise e
    return model

# Load dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi evaluasi ROUGE, F1 Score, dan ROC curve
def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)  # Prediksi model dalam bentuk logits
    probabilities = softmax(logits, dim=-1).cpu().numpy()  # Konversi logits ke probabilitas

    # Misal, jika model Anda adalah binary classification
    pred_probs = probabilities[:, 1]  # Probabilitas untuk kelas positif
    labels = pred.label_ids

    # Hitung ROC curve
    fpr, tpr, _ = roc_curve(labels, pred_probs)
    roc_auc = auc(fpr, tpr)

    # Hitung F1 Score
    pred_labels = pred_probs.round()  # Threshold 0.5 untuk binary classification
    f1 = f1_score(labels, pred_labels, average='weighted')

    # Hitung ROUGE (jika relevan, sesuaikan untuk task lain)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {key: 0 for key in scorer.score("", "").keys()}
    n = len(labels)

    for i in range(n):
        scores = scorer.score(str(labels[i]), str(pred_labels[i]))
        for key in rouge_scores:
            rouge_scores[key] += scores[key].fmeasure

    # Ambil rata-rata ROUGE score
    rouge_scores = {key: value / n for key, value in rouge_scores.items()}

    # Log metrik ke Neptune
    run["metrics/f1"].append(f1)
    run["metrics/roc_auc"].append(roc_auc)
    run["metrics/rouge1"].append(rouge_scores['rouge1'])
    run["metrics/rouge2"].append(rouge_scores['rouge2'])
    run["metrics/rougeL"].append(rouge_scores['rougeL'])

    # Plot ROC curve
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    
    # Simpan gambar ROC curve sebagai HTML interaktif
    run["visuals/roc_curve_interactive"].upload(File.as_html(plt))

    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy_score(labels, pred_labels),
        "f1": f1,
        "rouge1": rouge_scores['rouge1'],
        "rouge2": rouge_scores['rouge2'],
        "rougeL": rouge_scores['rougeL']
    }

# Modifikasi fungsi fine_tune_model untuk menyertakan tokenizer sebagai global variable
def fine_tune_model(model_name, dataset_file, output_dir):
    global tokenizer
    # Load model dan tokenizer
    model = load_model(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load dataset
    data = load_dataset(dataset_file)

    # Preprocess dataset
    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    # Logging hyperparameter ke Neptune
    params = {
        "model_name": model_name,
        "batch_size": 12,
        "num_train_epochs": 16,
        "learning_rate": 0.001,
        "gradient_accumulation_steps": 16,
    }
    run["parameters"] = params

    # Mendefinisikan argumen training
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=params["batch_size"],
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        num_train_epochs=params["num_train_epochs"],
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Menggunakan fungsi compute_metrics yang baru
    )

    # Jalankan pelatihan
    trainer.train()

    # Simpan model dan tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Hentikan logging Neptune
    run.stop()

    print(f"Model fine-tuning selesai dan disimpan di: {output_dir}")

# Contoh penggunaan
bucket_dataset_path = 'gs://pintarpath-trying/output2.json'  # Path di GCS
local_dataset_file = './output2.json'  # Path lokal untuk dataset
bucket_output_dir = 'gs://pintarpath-trying/saved_model/fine-tuned-model'  # Path output di GCS
local_output_dir = './fine-tuned-model'  # Path lokal untuk menyimpan model sebelum upload

# Download dataset dari GCS
download_from_gcs(bucket_dataset_path, local_dataset_file)

# Fine-tuning model
fine_tune_model(model_name="kalisai/Nusantara-0.8b-Indo-Chat", dataset_file=local_dataset_file, output_dir=local_output_dir)

# Upload model ke GCS (Dikomentari untuk menghemat penyimpanan)
# upload_to_gcs(local_output_dir, bucket_output_dir)


In [ ]:
import torch
import neptune
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
import os
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from torch.nn.functional import softmax
from neptune.types import File

# Inisialisasi run di Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),
    name="Fine-tuning with Real-time ROC",
    tags=["fine-tuning", "NLP", "Transformers", "real-time"],
    capture_hardware_metrics=True,
)

def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)
    probabilities = softmax(logits, dim=-1).cpu().numpy()

    pred_probs = probabilities[:, 1]  # Jika binary classification
    labels = pred.label_ids

    # Hitung ROC curve
    fpr, tpr, _ = roc_curve(labels, pred_probs)
    roc_auc = auc(fpr, tpr)

    # Log ROC curve sebagai interaktif HTML ke Neptune secara real-time
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Real-time ROC Curve')
    plt.legend(loc='best')
    
    # Unggah ROC curve ke Neptune sebagai HTML interaktif
    run["visuals/roc_curve"].upload(File.as_html(plt))

    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy_score(labels, pred_probs.round()),
        "f1": f1_score(labels, pred_probs.round(), average='weighted'),
    }

# Contoh pelatihan model dengan logging real-time
def fine_tune_model():
    model_name = "kalisai/Nusantara-0.8b-Indo-Chat"
    dataset_file = './output2.json'
    output_dir = './fine-tuned-model'
    
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    data = load_dataset(dataset_file)  # Fungsi ini memuat dataset dari file JSON

    # Preprocess dataset
    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=16,
        num_train_epochs=16,
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,  # Panggil fungsi compute_metrics yang telah diperbarui
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    run.stop()

fine_tune_model()


In [ ]:
import torch
import neptune
import os
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from torch.nn.functional import softmax
from neptune.types import File
import subprocess
import json

# Fungsi untuk memuat dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi untuk mengunggah file ke GCS
def upload_to_gcs(local_path, bucket_path):
    subprocess.run(['gsutil', 'cp', local_path, bucket_path])

# Inisialisasi Neptune dengan penanganan error
try:
    run = neptune.init_run(
        project="gabrielbatavia/Pintarpath",
        api_token=os.getenv("NEPTUNE_API_TOKEN"),
        name="Fine-tuning with Real-time ROC and Loss Visualization",
        tags=["fine-tuning", "NLP", "Transformers", "real-time"],
        capture_hardware_metrics=True,
    )
    neptune_connected = True
except Exception as e:
    print(f"Koneksi ke Neptune gagal: {e}")
    neptune_connected = False

def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)
    probabilities = softmax(logits, dim=-1).cpu().numpy()

    pred_probs = probabilities[:, 1]
    labels = pred.label_ids

    # Hitung ROC curve
    fpr, tpr, _ = roc_curve(labels, pred_probs)
    roc_auc = auc(fpr, tpr)

    # Visualisasi ROC curve
    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')

    # Jika terhubung ke Neptune, unggah visualisasi ke Neptune
    if neptune_connected:
        try:
            run["visuals/roc_curve"].upload(File.as_html(plt))
        except Exception as e:
            print(f"Gagal mengunggah ROC ke Neptune: {e}")
    else:
        # Jika tidak terhubung ke Neptune, simpan secara lokal dan upload ke GCS
        local_path = "roc_curve.html"
        plt.savefig(local_path)
        print(f"ROC Curve disimpan secara lokal: {local_path}")

        # Upload ke GCS
        bucket_path = 'gs://pintarpath-trying/roc_curve.html'
        upload_to_gcs(local_path, bucket_path)
        print(f"ROC Curve diunggah ke GCS: {bucket_path}")

    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy_score(labels, pred_probs.round()),
        "f1": f1_score(labels, pred_probs.round(), average='weighted'),
    }

# Fungsi fine_tune_model yang mencatat loss setiap batch
def fine_tune_model():
    model_name = "kalisai/Nusantara-0.8b-Indo-Chat"
    dataset_file = './output2.json'
    output_dir = './fine-tuned-model'
    
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    data = load_dataset(dataset_file)  # Panggil fungsi load_dataset

    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=16,
        num_train_epochs=16,
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,
    )

    # Modifikasi pelatihan untuk mencatat loss per batch
    loss_values = []
    for epoch in range(training_args.num_train_epochs):
        for step, batch in enumerate(dataset):
            model.train()
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            trainer.optimizer.step()
            trainer.lr_scheduler.step()
            trainer.optimizer.zero_grad()

            # Simpan loss
            loss_values.append(loss.item())

            # Log loss ke Neptune
            if neptune_connected:
                run[f"training/loss_per_batch"].append(loss.item())

    # Visualisasi Loss Curve
    plt.figure(figsize=(10, 7))
    plt.plot(loss_values, label='Training Loss per Batch')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.legend(loc='best')

    # Jika terhubung ke Neptune, unggah visualisasi ke Neptune
    if neptune_connected:
        run["visuals/loss_curve"].upload(File.as_html(plt))
    else:
        # Simpan loss curve secara lokal dan upload ke GCS
        loss_curve_path = "loss_curve.html"
        plt.savefig(loss_curve_path)
        print(f"Loss Curve disimpan secara lokal: {loss_curve_path}")

        # Upload ke GCS
        bucket_path = 'gs://pintarpath-trying/loss_curve.html'
        upload_to_gcs(loss_curve_path, bucket_path)
        print(f"Loss Curve diunggah ke GCS: {bucket_path}")

    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    if neptune_connected:
        run.stop()

fine_tune_model()


In [ ]:
import torch
import neptune
import os
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from torch.nn.functional import softmax
from neptune.types import File
import json
from rouge_score import rouge_scorer
from google.cloud import storage

# Inisialisasi run di Neptune
run = neptune.init_run(
    project="gabrielbatavia/Pintarpath",
    api_token=os.getenv("NEPTUNE_API_TOKEN"),
    name="Fine-tuning with Real-time ROC",
    tags=["fine-tuning", "NLP", "Transformers", "real-time"],
    capture_hardware_metrics=True,
)

# Inisialisasi GCP Storage client
storage_client = storage.Client()
bucket_name = "pintarpath-trying"  # Ganti dengan nama bucket GCP Anda
bucket = storage_client.bucket(bucket_name)

# Fungsi untuk memuat dataset dari file JSON
def load_dataset(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Fungsi compute_metrics dengan perhitungan ROUGE
def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)
    labels = pred.label_ids

    # Konversi prediksi dan label ke teks menggunakan tokenizer
    pred_texts = tokenizer.batch_decode(torch.argmax(logits, dim=-1), skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Menghitung ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(pred_text, label_text) for pred_text, label_text in zip(pred_texts, label_texts)]

    avg_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
    avg_rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
    avg_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

    # Visualisasi metrik ROUGE
    metrics = ['rouge1', 'rouge2', 'rougeL']
    values = [avg_rouge1, avg_rouge2, avg_rougeL]

    plt.figure(figsize=(10, 7))
    plt.bar(metrics, values, color='blue')
    plt.title('Average ROUGE Scores')
    plt.xlabel('Metrics')
    plt.ylabel('Scores')

    # Simpan visualisasi secara lokal dan upload ke GCP bucket
    plt.savefig('rouge_scores.png')
    run["visuals/rouge_scores"].upload(File.as_image('rouge_scores.png'))

    # Upload ke GCP bucket
    blob = bucket.blob('visualizations/rouge_scores.png')
    blob.upload_from_filename('rouge_scores.png')

    return {
        "rouge1": avg_rouge1,
        "rouge2": avg_rouge2,
        "rougeL": avg_rougeL,
    }

# Fungsi fine_tune_model yang memanggil load_dataset dan menghitung evaluation loss per batch
def fine_tune_model():
    model_name = "kalisai/Nusantara-0.8b-Indo-Chat"
    dataset_file = './output2.json'
    output_dir = './fine-tuned-model'
    
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    data = load_dataset(dataset_file)

    inputs = tokenizer([conv['conversations'][0]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    labels = tokenizer([conv['conversations'][1]['value'] for conv in data], return_tensors='pt', truncation=True, padding='max_length', max_length=64)

    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels['input_ids'][idx]
            return item

        def __len__(self):
            return len(self.labels['input_ids'])

    dataset = CustomDataset(inputs, labels)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=16,
        num_train_epochs=16,
        save_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=250,
        logging_dir='./logs',
        report_to="none",
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        compute_metrics=compute_metrics,
    )

    # Menambahkan callback untuk logging loss setiap batch
    def log_loss_callback(logs):
        run['train/loss'].log(logs['loss'])

    trainer.add_callback(log_loss_callback)

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    run.stop()

fine_tune_model()
